In [ ]:
library(Seurat)
library(stringr)
library(data.table)
library(tidyverse)
library(clusterProfiler)
library(org.Hs.eg.db)
library(ggplot2)
library(cowplot)
library(SingleR)
library(xlsx)
library(writexl)
library(dplyr)
library(ggrepel)
library(svglite)
library(Tempora)
library(ggalluvial)
library(RColorBrewer)
library(ggpubr)

library(Tempora)
library(Seurat)
library(RCurl)
library(tidyverse)
library(igraph)
library(ggraph)
library(graphlayouts)
library(ggforce)
library(scatterpie)
library(RColorBrewer)
library(igraph)
library(ggrepel)
library(stringr)
library(scales)



In [ ]:
save_plot = function(plotobj,outname, fig.width, fig.height)
{
  print(paste(outname, fig.width, fig.height))
  
  fname=paste(outname, "png", sep=".")
  print(paste("Saving to file", fname))
  png(filename=fname, width = fig.width, height = fig.height, units = 'in', res = 300)#width = fig.width*100, height=fig.height*100)
  plot(plotobj)
  dev.off()
  
  fname=paste(outname, "pdf", sep=".")
  print(paste("Saving to file", fname))
  pdf(file=fname, width = fig.width, height=fig.height)
  plot(plotobj)
  dev.off()
  

  fname=paste(outname, "svg", sep=".")
  print(paste("Saving to file", fname))
  svglite::svglite(file = fname, width = fig.width, height = fig.height)
  plot(plotobj)
  dev.off()
  
  return(plotobj)
}


In [ ]:
obj.bulk=readRDS("bulk_seurat_obj.rds")

In [ ]:
cellnames = names(obj.bulk$orig.ident)

mpoint = unlist(lapply(str_split(cellnames, "_"), function(x){return(x[3])}))
mpoint[mpoint == "tag4nl"] = "tag4"

mpoint_fact = factor(as.factor(mpoint), levels=c('ctrl','tag4', 'corkum','tag11','tag60'))


mpoint[mpoint == "tag4"] = "Day4"
mpoint[mpoint == "tag11"] = "Day11"
mpoint[mpoint == "tag60"] = "Day60"
mpoint[mpoint == "ctrl"] = "Ctrl"
mpoint[mpoint == "corkum"] = "Hosp."

mpoint2_fact = factor(as.factor(mpoint), levels=c('Ctrl','Day4', 'Hosp.','Day11','Day60'))

mpointcolors = as.vector(mpoint)
mpointcolors[mpointcolors == "Ctrl"] = "#1F1F1F"
mpointcolors[mpointcolors == "Hosp."] = "#E93A8D"
mpointcolors[mpointcolors == "Day4"] = "#069038"
mpointcolors[mpointcolors == "Day11"] = "#5DE223"
mpointcolors[mpointcolors == "Day60"] = "#CCCFCF"

mpoint2Colors = c("#1F1F1F", "#069038","#E93A8D","#5DE223","#CCCFCF")

obj.bulk$colors = mpointcolors

obj.bulk$mpoint = mpoint_fact
obj.bulk$mpoint2 = mpoint2_fact
obj.bulk$celltype = obj.bulk$orig.ident

obj.bulk$ct_time = paste(obj.bulk$celltype, obj.bulk$mpoint2)

In [ ]:
DimPlot(obj.bulk, reduction="pca", group.by="orig.ident")

In [ ]:
DimPlot(obj.bulk, reduction="pca", group.by="mpoint2", dims=c(2,3))

In [ ]:
DimPlot(obj.bulk, reduction="umap", group.by="orig.ident")

In [ ]:
p=DimPlot(obj.bulk, reduction="pca", group.by="orig.ident")
save_plot(p, "bulk_dimplots/pca_origident", 8, 8)

In [ ]:
p=DimPlot(obj.bulk, reduction="umap", group.by="orig.ident")
save_plot(p, "bulk_dimplots/umap_origident", 8, 8)

In [ ]:
p=DimPlot(obj.bulk, reduction="pca", group.by="mpoint2")+scale_color_manual(values = mpoint2Colors)
save_plot(p, "bulk_dimplots/pca_mpoint2", 8, 8)

In [ ]:
p=DimPlot(obj.bulk, reduction="umap", group.by="mpoint2")+scale_color_manual(values = mpoint2Colors)
save_plot(p, "bulk_dimplots/umap_mpoint2", 8, 8)

In [ ]:
obj.bulk = suppressWarnings(JackStraw(obj.bulk))

In [ ]:
obj.bulk = suppressWarnings(JackStraw(obj.bulk))
length(PCASigGenes(obj.bulk, pcs.use = c(2)))

In [ ]:
length(unique(interferonGenes))

In [ ]:
obj.bulk = suppressWarnings(JackStraw(obj.bulk))
length(PCASigGenes(obj.bulk, pcs.use = c(2)))
for (i in 1:30)
{
 pcaIgenes = PCASigGenes(obj.bulk, pcs.use = c(i), pval.cut=0.01, max.per.pc = 100)    
    
    print(paste(i, length(pcaIgenes), length(intersect(pcaIgenes, interferonGenes))))
}

In [ ]:
pcaGenes = PCASigGenes(obj.bulk, pcs.use = c(2), max.per.pc=500, pval.cut=0.001)

In [ ]:
p=DoHeatmap(obj.bulk, features = sort(pcaGenes), group.by="ct_time")
save_plot(p, "bulk_dimplots/pcagenes_heatmap", 20, 40)

In [ ]:
interferonGenes = c("MT2A", "ISG15", "LY6E", "IFIT1", "IFIT2", "IFIT3", "IFITM1", "IFITM3", "IFI44L", "IFI6", "MX1", "IFI27",  "IFI44L", "RSAD2", "SIGLEC1", "IFIT1", "ISG15")

In [ ]:
p=DoHeatmap(obj.bulk, features = sort(interferonGenes), group.by="ct_time")
save_plot(p, "bulk_dimplots/isg_heatmap", 15, 15)

In [ ]:
p=DotPlot(obj.bulk, features = sort(unique(interferonGenes)), group.by="ct_time")+ coord_flip()+  theme(axis.text.x = element_text(angle=90, vjust=.5, hjust=1))
save_plot(p, "bulk_dimplots/isg_dotplot", 15, 15)

In [ ]:
unique(obj.bulk$ctmp)

In [ ]:
plotUMAPDistr = function(selCelltype, title=NULL)
{
    obj.plot = subset(obj.bulk, orig.ident == selCelltype)
    print(obj.plot)
p=DimPlot(obj.plot, reduction="umap", group.by="mpoint2") + theme(legend.position="bottom")+labs(title=element_blank(),suptitle=element_blank())
p$data$ct_time = obj.plot$ct_time
p$data$colors = obj.plot$colors
p = p+scale_color_manual(values = mpoint2Colors)

dens1 <- ggplot(p$data, aes(x = UMAP_1, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none")

dens2 <- ggplot(p$data, aes(x = UMAP_2, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none") + 
  coord_flip()

mp = dens1 + plot_spacer() + p + dens2 + plot_layout(ncol = 2, nrow = 2, widths = c(4, 1), heights = c(1, 4))

    if (is.null(title))
        {
        title=selCelltype
    }
    
title <- ggdraw() + draw_label(title, fontface='bold')
p=cowplot::plot_grid(title, mp, ncol = 1, rel_heights = c(0.05, 1) )
    return(p)
}


for (celltype in unique(obj.bulk$orig.ident))
{
    p=plotUMAPDistr(celltype)
    save_plot(p, paste("bulk_dimplots/umap_", celltype, sep=""), 8, 8)
    plot(p)
}

In [ ]:
plotPCADistr = function(selCelltype, title=NULL)
{
    obj.plot = subset(obj.bulk, orig.ident == selCelltype)
    print(obj.plot)
p=DimPlot(obj.plot, reduction="pca", group.by="mpoint2") + theme(legend.position="bottom")+labs(title=element_blank(),suptitle=element_blank())
p$data$ct_time = obj.plot$ct_time
p$data$colors = obj.plot$colors
p = p+scale_color_manual(values = mpoint2Colors)

dens1 <- ggplot(p$data, aes(x = PC_1, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none")

dens2 <- ggplot(p$data, aes(x = PC_2, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none") + 
  coord_flip()

mp = dens1 + plot_spacer() + p + dens2 + plot_layout(ncol = 2, nrow = 2, widths = c(4, 1), heights = c(1, 4))

    if (is.null(title))
        {
        title=selCelltype
    }
    
title <- ggdraw() + draw_label(title, fontface='bold')
p=cowplot::plot_grid(title, mp, ncol = 1, rel_heights = c(0.05, 1) )
    return(p)
}


for (celltype in unique(obj.bulk$orig.ident))
{
    p=plotPCADistr(celltype)
    save_plot(p, paste("bulk_dimplots/pca_", celltype, sep=""), 8, 8)
    plot(p)
}

In [ ]:
library(ggpubr)
library(patchwork)

In [ ]:
obj.plot = obj.bulk
print(obj.plot)
p=DimPlot(obj.plot, reduction="pca", group.by="mpoint2", dims=c(2,3)) + theme(legend.position="bottom")+labs(title=element_blank(),suptitle=element_blank())
p$data$ct_time = obj.plot$ct_time
p$data$colors = obj.plot$colors
p = p+scale_color_manual(values = mpoint2Colors)

dens1 <- ggplot(p$data, aes(x = PC_2, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none")

dens2 <- ggplot(p$data, aes(x = PC_3, fill = mpoint2))+scale_fill_manual(values = mpoint2Colors) + 
  geom_density(alpha = 0.4) + 
  theme_void() + 
  theme(legend.position = "none") + 
  coord_flip()

mp = dens1 + plot_spacer() + p + dens2 + plot_layout(ncol = 2, nrow = 2, widths = c(4, 1), heights = c(1, 4))

    if (is.null(title))
        {
        title=selCelltype
    }
    
title <- ggdraw() + draw_label("", fontface='bold')
p=cowplot::plot_grid(title, mp, ncol = 1, rel_heights = c(0.05, 1) )

save_plot(p, paste("bulk_dimplots/pca_2_3", sep=""), 8, 8)


In [ ]:
p=DimPlot(obj.bulk, reduction="pca", group.by="orig.ident", dims=c(2,3))
save_plot(p, paste("bulk_dimplots/pca_origident_2_3", sep=""), 8, 8)
